In [143]:
reload(corrlab)


<module 'corrlab' from 'corrlab.py'>

In [38]:
cursor.execute("DROP TABLE IF EXISTS tb_corr")

0L

In [ ]:

# set parameters
filedir ='/hdd/ctp/day/'
start_date = '20171101'
end_date = '20171215'
type = 1      # 1 for aggravated, 0 for rolling
ticker1 = 'ni0'
outputdir = u'/home/hui/文档/corr output/'
lagLst = ['1s','5s','10s','30s','60s']
periodLst = ['1s','5s','10s','30s','60s']

# typelst = ['noble', 'nonferrous', 'black', 'farm', 'chemical', 'futures', 'loan']

import pandas as pd
import numpy as np
import re
import matplotlib.pylab as plt
import seaborn as sns
import os
import gc
import corrlab
import MySQLdb


def createTable():
    conn = MySQLdb.connect(host = 'localhost',user='root',passwd='hhui123456')
    cursor = conn.cursor()
    cursor.execute("""create database if not exists db_corr""")
    conn.select_db('db_corr')
    cursor.execute("""create table if not exists tb_corr(
    start_date DATE not null,
    end_date DATE not null,
    ticker1 varchar(32) not null,
    ticker2 varchar(32) not null,
    type SMALLINT not null DEFAULT 0, 
    period INT not null,
    lag INT not null,
    corr DOUBLE,
    symbol1 varchar(32),
    symbol2 varchar(32),
    primary key(start_date, end_date, ticker1, ticker2, type, lag)
    )""")

createTable() 


conn = MySQLdb.connect(host='localhost', user='root', passwd='hhui123456')
cursor = conn.cursor()
conn.select_db('db_corr')

res = pd.DataFrame()

corr = corrlab.corrAna(filedir = filedir, start_date= start_date, end_date= end_date, type= type)
dayLst = corr.generateDayLst()

for day in dayLst:   # 时间跨度为1天
    lst = []
    lst.append(day)
    data = corr.concatdata(lst)

    symbol1 = corr.symbolDict[day][ticker1[:2]]
    for lag in lagLst:
        for period in periodLst:
            res = pd.DataFrame()
            temp = data.copy()
            shifted = temp[ticker1].shift(-int(lag[:-1]), 's')
            align_base = corr.get_align_base(data)
            _, align_shifted = align_base.align(shifted, join='left', axis=0)
            temp[ticker1] = align_shifted.values
            temp = corr.sampledata(temp, period = period)
            temp.fillna(method='ffill',inplace=True)
            temp.fillna(method = 'bfill',inplace=True)
            temp_corr = temp.corr().sort_index()
            res = pd.concat([res,temp_corr[ticker1]])
            res.rename(columns = {0:day},inplace=True)
            res.fillna(-2,inplace=True)
            print 'lag is : %s, period is %s' %(lag, period)
            for ticker2 in temp_corr.index.values:
                corr_value = res[day][ticker2]
                ticker2 = ticker2.split('_')[0]
                symbol2 = corr.symbolDict[day][ticker2[:2]]
                cursor.execute("""REPLACE INTO tb_corr(
                            start_date,
                            end_date,
                            ticker1,
                            symbol1,
                            ticker2,
                            symbol2,
                            type,
                            period,
                            lag,
                            corr)
                            VALUES (
                            '%s', '%s','%s','%s','%s','%s','%d','%d','%d','%.6f'
                            )
                            """ % (day, day, ticker1, symbol1, ticker2, symbol2, type,int(period[:-1]), int(lag[:-1]), corr_value))
                conn.commit()

/home/hui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:28: Warning: Can't create database 'db_corr'; database exists
/home/hui/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:42: Warning: Table 'tb_corr' already exists
/home/hui/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/hui/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item

lag is : 1s, period is 1s
lag is : 1s, period is 5s
lag is : 1s, period is 10s
lag is : 1s, period is 30s
lag is : 1s, period is 60s
lag is : 5s, period is 1s
lag is : 5s, period is 5s
lag is : 5s, period is 10s
lag is : 5s, period is 30s
lag is : 5s, period is 60s
lag is : 10s, period is 1s
lag is : 10s, period is 5s
lag is : 10s, period is 10s
lag is : 10s, period is 30s
lag is : 10s, period is 60s
lag is : 30s, period is 1s
lag is : 30s, period is 5s
lag is : 30s, period is 10s
lag is : 30s, period is 30s
lag is : 30s, period is 60s
lag is : 60s, period is 1s
lag is : 60s, period is 5s
lag is : 60s, period is 10s
lag is : 60s, period is 30s
lag is : 60s, period is 60s
lag is : 1s, period is 1s
lag is : 1s, period is 5s
lag is : 1s, period is 10s
lag is : 1s, period is 30s
lag is : 1s, period is 60s
lag is : 5s, period is 1s
lag is : 5s, period is 5s
lag is : 5s, period is 10s
lag is : 5s, period is 30s
lag is : 5s, period is 60s
lag is : 10s, period is 1s
lag is : 10s, period is 5s


lag is : 1s, period is 30s
lag is : 1s, period is 60s
lag is : 5s, period is 1s
lag is : 5s, period is 5s
lag is : 5s, period is 10s
lag is : 5s, period is 30s
lag is : 5s, period is 60s
lag is : 10s, period is 1s
lag is : 10s, period is 5s
lag is : 10s, period is 10s
lag is : 10s, period is 30s
lag is : 10s, period is 60s
lag is : 30s, period is 1s
lag is : 30s, period is 5s
lag is : 30s, period is 10s
lag is : 30s, period is 30s
lag is : 30s, period is 60s
lag is : 60s, period is 1s
lag is : 60s, period is 5s
lag is : 60s, period is 10s
lag is : 60s, period is 30s
lag is : 60s, period is 60s
lag is : 1s, period is 1s
lag is : 1s, period is 5s
lag is : 1s, period is 10s
lag is : 1s, period is 30s
lag is : 1s, period is 60s
lag is : 5s, period is 1s
lag is : 5s, period is 5s
lag is : 5s, period is 10s
lag is : 5s, period is 30s
lag is : 5s, period is 60s
lag is : 10s, period is 1s
lag is : 10s, period is 5s
lag is : 10s, period is 10s
lag is : 10s, period is 30s
lag is : 10s, period is

In [47]:
temp_corr.index

Index([u'CF0', u'FG0', u'IC0', u'IF0', u'IH0', u'MA0', u'OI0', u'RM0', u'SF0',
       u'SM0', u'T10', u'TA0', u'TF0', u'ZC0', u'a10', u'ag0', u'al0', u'au0',
       u'bu0', u'c10', u'cs0', u'cu0', u'hc0', u'i10', u'j10', u'jd0', u'jm0',
       u'l10', u'm10', u'ni0', u'p10', u'pb0', u'pp0', u'rb0', u'ru0', u'sn0',
       u'v10', u'y10', u'zn0'],
      dtype='object')

In [45]:
cursor.execute('select * from tb_corr')


5L

In [46]:
rec = cursor.fetchall()
for i in rec:
    print i

(datetime.date(2017, 11, 13), datetime.date(2017, 11, 13), 'ru0', 'ru0', 1, 60L, 1L, 0.472855, 'ru1801', 'ru1801')
(datetime.date(2017, 11, 13), datetime.date(2017, 11, 13), 'ru0', 'ru0', 1, 60L, 5L, 0.46623, 'ru1801', 'ru1801')
(datetime.date(2017, 11, 13), datetime.date(2017, 11, 13), 'ru0', 'ru0', 1, 60L, 10L, 0.465898, 'ru1801', 'ru1801')
(datetime.date(2017, 11, 13), datetime.date(2017, 11, 13), 'ru0', 'ru0', 1, 60L, 30L, 0.465102, 'ru1801', 'ru1801')
(datetime.date(2017, 11, 13), datetime.date(2017, 11, 13), 'ru0', 'ru0', 1, 60L, 60L, 0.459677, 'ru1801', 'ru1801')


In [171]:
symbol

'SF0'